# WPC 만들기

In [ ]:
import pandas as pd

# 1. 엑셀 파일 경로
file_path = ".xlsx"  # 필요시 경로 수정

# 2. 시트 불러오기
df_work = pd.read_excel(file_path, sheet_name="Work")
df_pub = pd.read_excel(file_path, sheet_name="Publication")
df_col = pd.read_excel(file_path, sheet_name="Collection")
df_edge_pub_w = pd.read_excel(file_path, sheet_name="Edge_(Pub-W)")
df_edge_pub_c = pd.read_excel(file_path, sheet_name="Edge_(Pub-C)")

# 3. 병합용 원본 ID 보존
df_work["work_id_orig"] = df_work["id"]
df_pub["pub_id_orig"] = df_pub["id"]
df_col["col_id_orig"] = df_col["id"]

# 4. 컬럼 prefix 부여
df_work_prefixed = df_work.rename(columns=lambda x: f"work_{x}" if x != "work_id_orig" else x)
df_pub_prefixed = df_pub.rename(columns=lambda x: f"pub_{x}" if x != "pub_id_orig" else x)
df_col_prefixed = df_col.rename(columns=lambda x: f"col_{x}" if x != "col_id_orig" else x)

# 5. 엣지 병합
edge_pub_w = df_edge_pub_w.rename(columns={"source id": "pub_id", "target id": "work_id"})
edge_pub_c = df_edge_pub_c.rename(columns={"source id": "pub_id", "target id": "col_id"})
pub_core = pd.merge(edge_pub_c, edge_pub_w, on="pub_id", how="outer")

# 6. 정보 병합 (ID 기준, prefix 이전 ID 사용)
pub_core = pub_core.merge(df_pub_prefixed, left_on="pub_id", right_on="pub_id_orig", how="left")
pub_core = pub_core.merge(df_work_prefixed, left_on="work_id", right_on="work_id_orig", how="left")
pub_core = pub_core.merge(df_col_prefixed, left_on="col_id", right_on="col_id_orig", how="left")

# 7. 연결된 ID 목록 추출
linked_work_ids = pub_core['work_id_x'].dropna().unique().tolist()
linked_pub_ids = pub_core['pub_id_x'].dropna().unique().tolist()
linked_col_ids = pub_core['col_id_x'].dropna().unique().tolist()
# 8. 단독 항목 추출
solo_work = df_work_prefixed[~df_work_prefixed['work_id_orig'].isin(linked_work_ids)].copy()
solo_pub = df_pub_prefixed[~df_pub_prefixed['pub_id_orig'].isin(linked_pub_ids)].copy()
solo_col = df_col_prefixed[~df_col_prefixed['col_id_orig'].isin(linked_col_ids)].copy()

# 9. 최종 출력 컬럼 정의
final_columns = [
    'col_id_orig', 'col_name',
    'pub_id_orig', 'pub_name',
    'work_id_orig', 'work_korname', 'work_chiname',
    'work_titleExam', 'work_style', 'work_original', 'work_translation', 'work_url'
]

# 10. 누락 컬럼 보완 및 정렬
for df in [solo_work, solo_pub, solo_col]:
    for col in final_columns:
        if col not in df.columns:
            df[col] = None
    df = df[final_columns]

pub_core_final = pub_core[final_columns]

# 11. 전체 병합
full_merged = pd.concat([pub_core_final, solo_work, solo_pub, solo_col], ignore_index=True)

# 12. 저장
full_merged.to_csv("full_merged_with_prefix.csv", index=False)

### EPQA생성

In [32]:
import pandas as pd

# === 경로 설정 ===
epqa_path = "EPQA.xlsx"

# === 시트 로드 ===
df_person = pd.read_excel(epqa_path, sheet_name="Person").rename(columns={"id": "person_id"})
df_answer = pd.read_excel(epqa_path, sheet_name="Answer").rename(columns={"id": "answer_id"})
df_question = pd.read_excel(epqa_path, sheet_name="Question").rename(columns={"id": "question_id", "name": "Q_name"})
df_exam = pd.read_excel(epqa_path, sheet_name="Exam").rename(columns={"id": "exam_id", "name": "E_name"})

df_edge_pea = pd.read_excel(epqa_path, sheet_name="Edge_(Pe-A)").rename(columns={"source id": "person_id", "target id": "answer_id"}).dropna().drop_duplicates()
df_edge_aq = pd.read_excel(epqa_path, sheet_name="Edge_(A-Q)").rename(columns={"source id": "answer_id", "target id": "question_id"}).dropna().drop_duplicates()
df_edge_qe = pd.read_excel(epqa_path, sheet_name="Edge_(Q-E)").rename(columns={"source id": "question_id", "target id": "exam_id"}).dropna().drop_duplicates()

# === 단계 1: Person + Answer (via Edge) ===
df_pa = df_edge_pea.merge(df_person, on="person_id", how="left") \
                   .merge(df_answer, on="answer_id", how="left")
df_pa.to_excel("step1_PA.xlsx", index=False)
print("✅ Step 1 (PA) 저장:", df_pa.shape)

# === 단계 2: Question + Exam (via Edge) ===
df_qe = df_edge_qe.merge(df_question, on="question_id", how="left") \
                  .merge(df_exam, on="exam_id", how="left")
df_qe.to_excel("step2_QE.xlsx", index=False)
print("✅ Step 2 (QE) 저장:", df_qe.shape)

✅ Step 1 (PA) 저장: (1090, 21)
✅ Step 2 (QE) 저장: (940, 19)


In [29]:
df_epqa_flat.head()

,person_id,answer_id,question_id,exam_id,source name
0,PE0000,A17401005SS314,Q17401005SS314,E17401005SS3,1740년_증광_생원시_복시_문제
1,PE000000LBS,A18350808SF322,Q18350808SF322,E18350808SF322,1835년_증광_생원시_초시_종장_문제
2,PE000000SSG,A18310219SF122,Q18310219SF122,E18310219SF1,1831년_식년_진사시_초시_문제
3,PE000000SYR,A18350806SF112,Q18350806SF112,E18350806SF112,1835년_증광_진사시_초시_초장_문제
4,PE1153600JC,AS15610819,Q15610819,E15610819,1561년_식년시_생원_전시_문제


In [25]:
df_epqa_flat.to_excel("EPQA_flat.xlsx", index=True)

## flat 열 핸드크래프트로 정리, 복붙열 정리

In [21]:
import pandas as pd
import re

# 엑셀 파일 불러오기
df = pd.read_excel('EPQA_flat_revised.xlsx')  # 파일명 변경

# 정규식 패턴: 문자 1 + 숫자 3 + _ + 숫자 3 + 문자 1
pattern = r'[A-Za-z]\d{3}_\d{3}[A-Za-z]'

# Q_abstract 열에서 해당 패턴을 제거
df['Q_abstract'] = df['Q_abstract'].astype(str).apply(lambda x: re.sub(pattern, '', x))

# 결과 저장 (선택사항)
df.to_excel('EPQA_cleaned.xlsx', index=False)

In [22]:
import pandas as pd

# 데이터 불러오기
df = pd.read_excel('EPQA_cleaned.xlsx')  # 실제 파일명으로 변경
duplicated_rows = df[df.duplicated()]
print(f"중복 행 개수: {len(duplicated_rows)}")

중복 행 개수: 0
